<a href="https://colab.research.google.com/github/WaterPurify/Text_Mining/blob/main/Group_Project/Copy_of_Hotel_review3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
path = '/content/'
d1 = pd.read_csv(path + '아고다크롤링_리뷰랑날짜.csv')
d2 = pd.read_csv(path + '아고다크롤링_국적객실유형.csv')
d3 = pd.read_csv(path + '아고다크롤링_평점이랑여행유형.csv')
# 데이터프레임을 열 방향으로 합치기
df = pd.concat([d1, d2, d3], axis=1)
df_original = df.copy()

In [3]:
df.to_csv('/content/아고다크롤링_리뷰.csv')

##정규 표현식


*   리뷰들 중에서 한국어,영어,일본어 이렇게 3개의 언어로 구성되어 있습니다.

    *   하지만 영어로된 리뷰들이 압도적으로 많기 때문에 영어 리뷰들과 숫자들을 뽑기로 결정하였습니다.
    *   그리고 만약 번역을 한다면 의미가 변질될 수 있기 때문에 번역을 하지 않기로 했습니다


In [4]:
#소문자로  만들어주는 함수
def make_lower_case(series):
    """
    Convert all strings in the input Pandas Series to lowercase.

    Parameters:
    series (pandas.Series): Input Series containing strings.

    Returns:
    pandas.Series: New Series with strings converted to lowercase.
    """
    return series.str.lower()

In [5]:
import re

#리뷰들을 review로 받음


#통일을 하기 위해 다 소문자로 바꿉니다
review_content= df['review_content']

review_lowercase=make_lower_case(review_content)
review_lowercase


0       is it necessary to write a review for this hot...
1       as expected, shilla is shilla!! adding the eve...
2       exellent location & service”\nwe stayed at shi...
3       "i can understand why many people are looking ...
4       완벽산 신라호텔 경험”\n가든뷰에 17개월 아기와 함께 묵었습니다. 비록 저녁에 윗...
                              ...                        
2595    a very nice resort”\nmake you feel great once ...
2596    the hotel rooms update is way over due.”\nthe ...
2597    hotel with great children facilities”\nthis ho...
2598    perfect hotel for relaxing”\nthe hotel offers ...
2599    would definitely go there again”\none of my fa...
Name: review_content, Length: 2600, dtype: object

In [6]:
def remove_newline(series):
    # 문자열에서 '\n'을 제거하고 반환
    return series.str.replace('\n', '')
Remove_n = remove_newline(review_lowercase)
Remove_n

0       is it necessary to write a review for this hot...
1       as expected, shilla is shilla!! adding the eve...
2       exellent location & service”we stayed at shill...
3       "i can understand why many people are looking ...
4       완벽산 신라호텔 경험”가든뷰에 17개월 아기와 함께 묵었습니다. 비록 저녁에 윗층에...
                              ...                        
2595    a very nice resort”make you feel great once yo...
2596    the hotel rooms update is way over due.”the sh...
2597    hotel with great children facilities”this hote...
2598    perfect hotel for relaxing”the hotel offers ma...
2599    would definitely go there again”one of my favo...
Name: review_content, Length: 2600, dtype: object

In [7]:

def Extract_number_alphabet(series):
    """
    Extract numbers and words from the given strings

    Parameters:
    series (pandas.Series): Input Series containing strings with symbols.

    Returns:
    pandas.Series: New Series with symbols replaced with spaces.
    """
    extracted_series = series.str.extractall(r'([a-zA-Z0-9]+)').groupby(level=0).agg(' '.join)
    return extracted_series

In [8]:
#\n이 붙어 있는 글자가 있기 때문에 space로 바꿔줍니다

review_preprocessed = Extract_number_alphabet(Remove_n)


print(type(review_preprocessed))
review_preprocessed[0]

<class 'pandas.core.frame.DataFrame'>


0       is it necessary to write a review for this hot...
1       as expected shilla is shilla adding the evenin...
2       exellent location service we stayed at shilla ...
3       i can understand why many people are looking f...
4                                                      17
                              ...                        
2595    a very nice resort make you feel great once yo...
2596    the hotel rooms update is way over due the shi...
2597    hotel with great children facilities this hote...
2598    perfect hotel for relaxing the hotel offers ma...
2599    would definitely go there again one of my favo...
Name: 0, Length: 2513, dtype: object

In [9]:
unique_series = review_preprocessed[0].unique()
unique_series

array(['is it necessary to write a review for this hotel they upgraded my room so i decided to write a review haha this is the best hotel that everyone in korea knows about with service and facilities that match its reputation the staff are friendly even though i checked in late i still got a room upgrade and the interior design turns the old into vintage but it s not outdated at all during my stay i walked around the well maintained gardens near the hotel after having breakfast and it was the happiest memory of my jeju trip even if there are many new and good hotels shilla is definitely the best in jeju i plan to go again next year i need to make more money',
       'as expected shilla is shilla adding the evening view at the parkview made it completely satisfying during the room change the female staff member who helped us was very kind and helpful unfortunately we couldn t see her name because her jacket was covering it she was really great all the other staff members were also very

##언어 분류 및 번역
* 호텔  리뷰 데이터에서 영어,스페인어,프랑스어,일본어 다양한 리뷰들이 존재합니다. 더 많은 데이터를 얻기 위해서 영어로 다 변역을 진행시키는 과정이 필요합니다.

* 번역을 해도 우리 프로젝트에 영향을 주지 않는 이유는 키워드로 추출하기 때문입니다.

In [10]:
#Landetect를 통해서 일단 다른 언어인지 분류하는것 필요함
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993227 sha256=ec12ab95aa9ecedb0543567c035a79de1e8336184bd564c14ea0512b8b16b674
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [11]:
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException
dict ={}
def detect_language(text):
    try:
        language = detect(text)
        return language
    except LangDetectException as e:
        print("Error:", e)
        return None

for lang in unique_series:
    dict[lang]=detect_language(lang)
    # print(detect_language(lang))

print(dict)

Error: No features in text.
Error: No features in text.
{'is it necessary to write a review for this hotel they upgraded my room so i decided to write a review haha this is the best hotel that everyone in korea knows about with service and facilities that match its reputation the staff are friendly even though i checked in late i still got a room upgrade and the interior design turns the old into vintage but it s not outdated at all during my stay i walked around the well maintained gardens near the hotel after having breakfast and it was the happiest memory of my jeju trip even if there are many new and good hotels shilla is definitely the best in jeju i plan to go again next year i need to make more money': 'en', 'as expected shilla is shilla adding the evening view at the parkview made it completely satisfying during the room change the female staff member who helped us was very kind and helpful unfortunately we couldn t see her name because her jacket was covering it she was really

In [12]:
count = sum(1 for key, value in dict.items() if value == 'en')

print("Number of occurrences of 'en':", count)

Number of occurrences of 'en': 110


* 영어로된 문장만 사용하기 때문에 영어를 따로 추출

In [13]:
def get_keys_by_value(dictionary, target_value):
    keys = []
    for key, value in dictionary.items():
        if value == target_value:
            keys.append(key)
    return keys

In [14]:
target_value = 'en'
English_preprocessed = get_keys_by_value(dict,target_value)
English_preprocessed

['is it necessary to write a review for this hotel they upgraded my room so i decided to write a review haha this is the best hotel that everyone in korea knows about with service and facilities that match its reputation the staff are friendly even though i checked in late i still got a room upgrade and the interior design turns the old into vintage but it s not outdated at all during my stay i walked around the well maintained gardens near the hotel after having breakfast and it was the happiest memory of my jeju trip even if there are many new and good hotels shilla is definitely the best in jeju i plan to go again next year i need to make more money',
 'as expected shilla is shilla adding the evening view at the parkview made it completely satisfying during the room change the female staff member who helped us was very kind and helpful unfortunately we couldn t see her name because her jacket was covering it she was really great all the other staff members were also very kind howeve

NLTK을 이용한 토큰화


*   Naturak Language Toolkit(NLTK)는 자연어 처리를 위해 만들어진 파이썬 라이브러리입니다.
*  NLTK는 word_tokenize()를 통해 단어 토큰화를 할 수 있고 sent_tokenize()를 통해 문장 토큰화를 할 수 있습니다.




단어 토큰화

In [15]:
#NLTK 라이브러리 설치
!pip install --user -U nltk
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [16]:

from nltk.tokenize import word_tokenize
import numpy as np

def tokenize_text(text):
    return word_tokenize(text)

# Apply tokenization to each element in the array
tokenized_list = [tokenize_text(text) for text in English_preprocessed]

print(tokenized_list)

# nd array를 Series으로 변환
Tokeninzed_words =  pd.Series(tokenized_list)
Tokeninzed_words

[['is', 'it', 'necessary', 'to', 'write', 'a', 'review', 'for', 'this', 'hotel', 'they', 'upgraded', 'my', 'room', 'so', 'i', 'decided', 'to', 'write', 'a', 'review', 'haha', 'this', 'is', 'the', 'best', 'hotel', 'that', 'everyone', 'in', 'korea', 'knows', 'about', 'with', 'service', 'and', 'facilities', 'that', 'match', 'its', 'reputation', 'the', 'staff', 'are', 'friendly', 'even', 'though', 'i', 'checked', 'in', 'late', 'i', 'still', 'got', 'a', 'room', 'upgrade', 'and', 'the', 'interior', 'design', 'turns', 'the', 'old', 'into', 'vintage', 'but', 'it', 's', 'not', 'outdated', 'at', 'all', 'during', 'my', 'stay', 'i', 'walked', 'around', 'the', 'well', 'maintained', 'gardens', 'near', 'the', 'hotel', 'after', 'having', 'breakfast', 'and', 'it', 'was', 'the', 'happiest', 'memory', 'of', 'my', 'jeju', 'trip', 'even', 'if', 'there', 'are', 'many', 'new', 'and', 'good', 'hotels', 'shilla', 'is', 'definitely', 'the', 'best', 'in', 'jeju', 'i', 'plan', 'to', 'go', 'again', 'next', 'year',

0      [is, it, necessary, to, write, a, review, for,...
1      [as, expected, shilla, is, shilla, adding, the...
2      [exellent, location, service, we, stayed, at, ...
3      [i, can, understand, why, many, people, are, l...
4      [worth, every, penny, room, is, spacious, clea...
                             ...                        
105    [a, very, nice, resort, make, you, feel, great...
106    [the, hotel, rooms, update, is, way, over, due...
107    [hotel, with, great, children, facilities, thi...
108    [perfect, hotel, for, relaxing, the, hotel, of...
109    [would, definitely, go, there, again, one, of,...
Length: 110, dtype: object

##불용어 제거(Stopwords)



* 갖고 있는 데이터에서 유의미한 단어 토큰만을 선별하기 위해서는 큰 의미가 없는 단어 토큰을 제거하는 작업이 필요

* 큰 의미가 없다라는 것: 자주 등장하지만 분석을 하는 것에 있어서는 큰 도움이 되지 않는 단어들

* 예를 들면, I, my, me, over, 조사, 접미사 같은 단어들은 문장에서는 자주 등장하지만 실제 의미 분석을 하는데는 거의 기여하는 바가 없는 경우가 있습니다. 이러한 단어들을 불용어(stopword)라고 하며, NLTK에서는 위와 같은 100여개 이상의 영어 단어들을 불용어로 패키지 내에서 미리 정의

* 물론 불용어는 개발자가 직접 정의할 수도 있습니다.

#1. NLTK에서 불용어 확인하기

*영어의 불용어를 확인하여 저희의 데이터에 최적화하는 작업이 필요함
*다양한 불용어 package가 존재합니다


1.   Gensim
2.   Scikit-learn
3.   spacy
4.   NLTK



In [17]:
from nltk.tokenize import word_tokenize
import numpy as np
def tokenize_text(text):
    return word_tokenize(text)

# Apply tokenization to each element in the array
tokenized_list = [tokenize_text(text) for text in English_preprocessed]

In [18]:
#Gensim stopwords 와 개수
stopwords_Gensim =['twelve', 'yourselves', 'nor', 'con', 'here', 'our', 'whatever', 'mine', 'then', 'down', 'between', 'everyone', 'eg', 'just', 'yours', 'bill', 'none', 'inc', 'itself', 'be', 'because', 'hereupon', 'toward', 'me', 'six', 'some', 'noone', 'herself', 'in', 'above', 'make', 'thru', 'whereas', 'those', 'hence', 'himself', 'them', 'unless', 'yet', 'couldnt', 'last', 'under', 'he', 'somehow', 'or', 'neither', 'five', 'thereupon', 'part', 'two', 'full', 'using', 'up', 'really', 'wherever', 'as', 'front', 'back', 'nine', 'otherwise', 'sometime', 'after', 'seem', 'someone', 'anything', 'did', 'am', 'towards', 'whom', 'across', 'from', 'whence', 'less', 'wherein', 'have', 'four', 'quite', 'without', 'per', 'and', 'its', 'didn', 'been', 'hereafter', 'before', 'interest', 'please', 'seems', 'nowhere', 'very', 'well', 'but', 'whereafter', 'cannot', 'amongst', 'must', 'namely', 'few', 'hundred', 'behind', 'others', 'whenever', 'serious', 'among', 'even', 're', 'moreover', 'upon', 'of', 'him', 'anyway', 'show', 'alone', 'call', 'to', 'keep', 'since', 'still', 'onto', 'doesn', 'other', 'too', 'un', 'would', 'if', 'describe', 'next', 'had', 'along', 'fire', 'go', 'all', 'my', 'afterwards', 'this', 'so', 'has', 'further', 'seeming', 'sometimes', 'why', 'were', 'there', 'thereby', 'least', 'again', 'they', 'almost', 'several', 'top', 'ltd', 'than', 'another', 'enough', 'beforehand', 'at', 'else', 'get', 'becoming', 'for', 'an', 'various', 'ten', 'nevertheless', 'etc', 'throughout', 'latterly', 'ie', 'mostly', 'don', 'amoungst', 'beyond', 'beside', 'same', 'such', 'nobody', 'below', 'may', 'can', 'thence', 'about', 'out', 'cant', 'that', 'much', 'used', 'perhaps', 'already', 'indeed', 'detail', 'computer', 'does', 'should', 'take', 'which', 'yourself', 'hasnt', 'every', 'also', 'being', 'although', 'thick', 'whereupon', 'name', 'doing', 'each', 'fifteen', 'thereafter', 'with', 'only', 'ever', 'see', 'most', 'own', 'formerly', 'rather', 'de', 'you', 'us', 'she', 'we', 'km', 'when', 'during', 'eleven', 'however', 'regarding', 'against', 'first', 'are', 'together', 'once', 'on', 'often', 'always', 'whether', 'becomes', 'not', 'became', 'anywhere', 'though', 'was', 'sixty', 'who', 'the', 'co', 'besides', 'any', 'never', 'give', 'former', 'around', 'both', 'amount', 'more', 'themselves', 'one', 'empty', 'thin', 'whose', 'by', 'whither', 'could', 'will', 'find', 'whole', 'hers', 'say', 'therefore', 'hereby', 'fill', 'these', 'done', 'through', 'thus', 'her', 'sincere', 'until', 'what', 'might', 'how', 'his', 'latter', 'meanwhile', 'nothing', 'a', 'move', 'side', 'somewhere', 'their', 'is', 'found', 'become', 'elsewhere', 'over', 'ourselves', 'i', 'put', 'no', 'whereby', 'anyone', 'cry', 'eight', 'myself', 'ours', 'everything', 'except', 'forty', 'it', 'your', 'system', 'many', 'off', 'where', 'while', 'within', 'due', 'now', 'third', 'herein', 'twenty', 'whoever', 'into', 'do', 'bottom', 'everywhere', 'something', 'via', 'either', 'fify', 'seemed', 'anyhow', 'three', 'kg', 'therein', 'made', 'mill']

# 리스트 컴프리헨션을 사용하여 모든 리스트를 하나로 통합
# tokenized_list = [item for sublist in tokenized_list for item in sublist]


In [19]:



# 불용어를 제거한 결과를 저장할 리스트 초기화
filtered_data = []
# 각 리스트의 요소에서 불용어를 제거하고 결과를 새로운 리스트에 추가
for sublist in tokenized_list:
    filtered_sublist = [word for word in sublist if word not in stopwords_Gensim]
    filtered_data.append(filtered_sublist)
print(filtered_data)




# print('불용어 개수 :', len(stopwords))



[['necessary', 'write', 'review', 'hotel', 'upgraded', 'room', 'decided', 'write', 'review', 'haha', 'best', 'hotel', 'korea', 'knows', 'service', 'facilities', 'match', 'reputation', 'staff', 'friendly', 'checked', 'late', 'got', 'room', 'upgrade', 'interior', 'design', 'turns', 'old', 'vintage', 's', 'outdated', 'stay', 'walked', 'maintained', 'gardens', 'near', 'hotel', 'having', 'breakfast', 'happiest', 'memory', 'jeju', 'trip', 'new', 'good', 'hotels', 'shilla', 'definitely', 'best', 'jeju', 'plan', 'year', 'need', 'money'], ['expected', 'shilla', 'shilla', 'adding', 'evening', 'view', 'parkview', 'completely', 'satisfying', 'room', 'change', 'female', 'staff', 'member', 'helped', 'kind', 'helpful', 'unfortunately', 'couldn', 't', 'jacket', 'covering', 'great', 'staff', 'members', 'kind', 'women', 'charge', 'entrance', 'parkview', 'november', '14th', 'new', 'stayed', 'shilla', 'hotel', 'times', 'seen', 'unresponsive', 'overall', 'satisfied', 'come'], ['exellent', 'location', 'serv

In [20]:
#Scikit_learn stopwords 와 개수
stop_words_list_scikit_learn =frozenset({'already', 'ltd', 'until', 'on', 'besides', 'becoming', 'de', 'few', 'same', 'which', 'in', 'made', 'whom', 'though', 'being', 'front', 'whereby', 'whose', 'who', 'along', 'anyone', 'have', 'now', 'been', 'often', 'else', 'her', 'formerly', 'found', 'co', 'ever', 'can', 'mill', 'yet', 'hereby', 'always', 'around', 'only', 'eg', 'its', 'thereupon', 'there', 'at', 'nevertheless', 'mine', 'nine', 'give', 'becomes', 'hers', 'see', 'wherein', 'neither', 'nobody', 'sometime', 'six', 'too', 'behind', 'someone', 'himself', 'she', 'down', 'afterwards', 'about', 'with', 'anyway', 'fire', 'couldnt', 'they', 'their', 'first', 'somehow', 'un', 'herein', 'well', 'less', 'elsewhere', 'the', 'cannot', 'myself', 'would', 'therefore', 'thick', 'ours', 'some', 'empty', 'our', 'again', 'con', 'name', 'bottom', 'five', 'interest', 'were', 'me', 'may', 'next', 'per', 'seem', 'through', 'almost', 'below', 'from', 'thru', 'to', 'of', 'twenty', 'both', 'cry', 'every', 'somewhere', 'hence', 'under', 'cant', 'why', 'everyone', 'among', 'fill', 'is', 'amoungst', 'these', 'fifteen', 'become', 'perhaps', 'even', 'etc', 'part', 'than', 'ie', 'much', 'third', 'indeed', 'was', 'yours', 'any', 'another', 'together', 'whole', 'each', 'if', 'show', 'them', 'whatever', 'describe', 'whereafter', 'eleven', 'and', 'thereafter', 'has', 'three', 'it', 'herself', 'amount', 'up', 'everything', 'before', 'sincere', 'him', 'how', 'that', 'call', 'off', 'seemed', 'we', 'yourself', 'noone', 'while', 'into', 'very', 'several', 'across', 'above', 'be', 'but', 'not', 'could', 'therein', 'ourselves', 'due', 'once', 'hereafter', 'beside', 'hundred', 'no', 'for', 'keep', 'one', 'upon', 'after', 'other', 'or', 'amongst', 'last', 'others', 'serious', 'this', 'a', 'where', 'meanwhile', 'wherever', 'whether', 'mostly', 'an', 'full', 'forty', 'via', 'take', 'eight', 'side', 'you', 'anything', 'more', 'nothing', 'between', 'inc', 'move', 'whoever', 'never', 'am', 'because', 'beyond', 'either', 'except', 'former', 'namely', 'please', 'should', 'without', 'those', 'however', 'two', 'will', 'also', 'done', 'many', 'nor', 'out', 'detail', 'enough', 'thin', 'your', 'latterly', 'whence', 'as', 'yourselves', 'since', 'seems', 'thence', 'put', 'had', 'towards', 'moreover', 'throughout', 'further', 'so', 'toward', 'twelve', 'bill', 'during', 'against', 'still', 'alone', 'us', 'might', 'here', 'whereupon', 'nowhere', 'anywhere', 'within', 'hereupon', 'sometimes', 'became', 'over', 'i', 'onto', 'what', 're', 'although', 'he', 'itself', 'rather', 'latter', 'my', 'go', 'most', 'top', 'seeming', 'fifty', 'his', 'back', 'by', 'own', 'ten', 'then', 'must', 'when', 'none', 'everywhere', 'get', 'are', 'hasnt', 'least', 'system', 'sixty', 'beforehand', 'find', 'such', 'whither', 'anyhow', 'themselves', 'thus', 'something', 'whenever', 'four', 'thereby', 'all', 'whereas', 'otherwise', 'do'})

print('불용어 개수 :', len(stop_words_list_scikit_learn))

# 불용어를 제거한 결과를 저장할 리스트 초기화
Stop_words_Scikit_learn = []
# 각 리스트의 요소에서 불용어를 제거하고 결과를 새로운 리스트에 추가
for sublist in tokenized_list:
    filtered_sublist = [word for word in sublist if word not in list(stop_words_list_scikit_learn)]
    Stop_words_Scikit_learn.append(filtered_sublist)
print(Stop_words_Scikit_learn)


불용어 개수 : 318
[['necessary', 'write', 'review', 'hotel', 'upgraded', 'room', 'decided', 'write', 'review', 'haha', 'best', 'hotel', 'korea', 'knows', 'service', 'facilities', 'match', 'reputation', 'staff', 'friendly', 'checked', 'late', 'got', 'room', 'upgrade', 'interior', 'design', 'turns', 'old', 'vintage', 's', 'outdated', 'stay', 'walked', 'maintained', 'gardens', 'near', 'hotel', 'having', 'breakfast', 'happiest', 'memory', 'jeju', 'trip', 'new', 'good', 'hotels', 'shilla', 'definitely', 'best', 'jeju', 'plan', 'year', 'need', 'make', 'money'], ['expected', 'shilla', 'shilla', 'adding', 'evening', 'view', 'parkview', 'completely', 'satisfying', 'room', 'change', 'female', 'staff', 'member', 'helped', 'kind', 'helpful', 'unfortunately', 'couldn', 't', 'jacket', 'covering', 'really', 'great', 'staff', 'members', 'kind', 'women', 'charge', 'entrance', 'parkview', 'november', '14th', 'new', 'stayed', 'shilla', 'hotel', 'times', 'seen', 'unresponsive', 'overall', 'satisfied', 'come'],

In [21]:
#Spacy stopwords 와 개수
stop_words_list_spacy ={'why', 'does', 'latterly', 'mostly', 'but', 'otherwise', 'during', 'ourselves', 'forty', 'yourselves', 'hereupon', 'only', 'into', 'she', 'become', 'can', 'two', 'ten', 'ever', 'within', 'call', 'go', 'hundred', 'under', 'up', 'thru', 'well', 'nobody', 'is', 'noone', 'elsewhere', 'must', 'nothing', 'own', 'such', 'thereafter', 'me', 'any', 'several', 'behind', 'beside', 'his', 'your', 'have', 'therein', 'until', 'itself', 'should', 'moreover', 'everything', 'been', 'if', 'cannot', 'show', 'thereupon', 'nowhere', 'this', 'who', 'fifty', 'amount', 'an', 'however', 'since', 'as', 'that', 'whither', 'below', 'please', 'all', 'do', 'anything', 'else', 'used', 'very', 'third', 'name', 'various', 'either', 'i', 'our', 'when', 'will', 'sometimes', 'front', 'done', 'thus', 'herein', 'seemed', 'yet', 'indeed', 'whose', 'least', 'serious', 'last', 'still', 'much', 'another', 'along', 'him', 'nevertheless', 'again', 'many', 'none', 'he', 'nor', 'quite', 'alone', 'although', 'mine', 'same', 'ours', 'take', 'twelve', 'yours', 'meanwhile', 'full', 'due', 'its', 'hereafter', 'thereby', 'becomes', 'has', 'thence', 'toward', 'by', 'himself', 'never', 'fifteen', 'wherein', 'in', 'besides', 'whereby', 'be', 'each', 'could', 'from', 'together', 'would', 'hereby', 'whereafter', 'whether', 'down', 'eleven', 'formerly', 'were', 'and', 'top', 'others', 'see', 'whole', 'less', 'my', 'anyway', 'even', 'keep', 'both', 'over', 're', 'ca', 'side', 'there', 'namely', 'regarding', 'how', 'anyhow', 'upon', 'bottom', 'further', 'wherever', 'hence', 'then', 'more', 'across', 'against', 'five', 'somehow', 'herself', 'whenever', 'someone', 'them', 'had', 'often', 'yourself', 'seem', 'among', 'themselves', 'put', 'are', 'out', 'whoever', 'myself', 'enough', 'doing', 'beyond', 'off', 'those', 'while', 'no', 'onto', 'say', 'just', 'became', 'now', 'make', 'may', 'what', 'made', 'after', 'above', 'part', 'neither', 'everywhere', 'back', 'four', 'hers', 'per', 'first', 'perhaps', 'also', 'too', 'to', 'where', 'without', 'you', 'every', 'afterwards', 'seems', 'on', 'they', 'using', 'becoming', 'around', 'did', 'us', 'except', 'with', 'three', 'through', 'somewhere', 'seeming', 'a', 'towards', 'really', 'was', 'than', 'whatever', 'already', 'always', 'beforehand', 'because', 'anywhere', 'whereupon', 'rather', 'anyone', 'other', 'six', 'sixty', 'being', 'these', 'which', 'her', 'once', 'here', 'about', 'at', 'eight', 'some', 'for', 'latter', 'between', 'whereas', 'former', 'we', 'of', 'the', 'whence', 'something', 'so', 'it', 'most', 'few', 'unless', 'give', 'almost', 'twenty', 'amongst', 'before', 'sometime', 'or', 'their', 'not', 'therefore', 'nine', 'get', 'one', 'empty', 'move', 'whom', 'everyone', 'via', 'throughout', 'though', 'might', 'next', 'am'}

print('불용어 개수 :', len(stop_words_list_spacy))
# 불용어를 제거한 결과를 저장할 리스트 초기화
Stop_words_Spacy = []
# 각 리스트의 요소에서 불용어를 제거하고 결과를 새로운 리스트에 추가
for sublist in tokenized_list:
    filtered_sublist = [word for word in sublist if word not in list(stop_words_list_spacy)]
    Stop_words_Spacy.append(filtered_sublist)
print(Stop_words_Spacy)

불용어 개수 : 305
[['necessary', 'write', 'review', 'hotel', 'upgraded', 'room', 'decided', 'write', 'review', 'haha', 'best', 'hotel', 'korea', 'knows', 'service', 'facilities', 'match', 'reputation', 'staff', 'friendly', 'checked', 'late', 'got', 'room', 'upgrade', 'interior', 'design', 'turns', 'old', 'vintage', 's', 'outdated', 'stay', 'walked', 'maintained', 'gardens', 'near', 'hotel', 'having', 'breakfast', 'happiest', 'memory', 'jeju', 'trip', 'new', 'good', 'hotels', 'shilla', 'definitely', 'best', 'jeju', 'plan', 'year', 'need', 'money'], ['expected', 'shilla', 'shilla', 'adding', 'evening', 'view', 'parkview', 'completely', 'satisfying', 'room', 'change', 'female', 'staff', 'member', 'helped', 'kind', 'helpful', 'unfortunately', 'couldn', 't', 'jacket', 'covering', 'great', 'staff', 'members', 'kind', 'women', 'charge', 'entrance', 'parkview', 'november', '14th', 'new', 'stayed', 'shilla', 'hotel', 'times', 'seen', 'unresponsive', 'overall', 'satisfied', 'come'], ['exellent', 'loc

In [22]:
#NLTK stopwords 와 개수
stop_words_NLTK =['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

print('불용어 개수 :', len(stop_words_NLTK))

# 불용어를 제거한 결과를 저장할 리스트 초기화
Stop_words_NLTK = []
# 각 리스트의 요소에서 불용어를 제거하고 결과를 새로운 리스트에 추가
for sublist in tokenized_list:
    filtered_sublist = [word for word in sublist if word not in stop_words_NLTK]
    Stop_words_NLTK.append(filtered_sublist)
print(Stop_words_NLTK)

불용어 개수 : 179
[['necessary', 'write', 'review', 'hotel', 'upgraded', 'room', 'decided', 'write', 'review', 'haha', 'best', 'hotel', 'everyone', 'korea', 'knows', 'service', 'facilities', 'match', 'reputation', 'staff', 'friendly', 'even', 'though', 'checked', 'late', 'still', 'got', 'room', 'upgrade', 'interior', 'design', 'turns', 'old', 'vintage', 'outdated', 'stay', 'walked', 'around', 'well', 'maintained', 'gardens', 'near', 'hotel', 'breakfast', 'happiest', 'memory', 'jeju', 'trip', 'even', 'many', 'new', 'good', 'hotels', 'shilla', 'definitely', 'best', 'jeju', 'plan', 'go', 'next', 'year', 'need', 'make', 'money'], ['expected', 'shilla', 'shilla', 'adding', 'evening', 'view', 'parkview', 'made', 'completely', 'satisfying', 'room', 'change', 'female', 'staff', 'member', 'helped', 'us', 'kind', 'helpful', 'unfortunately', 'see', 'name', 'jacket', 'covering', 'really', 'great', 'staff', 'members', 'also', 'kind', 'however', 'two', 'women', 'charge', 'entrance', 'parkview', 'november

In [ ]:
df_original['tokenized_list']=Tokeninzed_words


,review_content,date_text,reviewer_country,room_type,review_score,group,tokenized_list
0,Is it necessary to write a review for this hot...,Stayed 2 nights in November 2023,Yumi from South Korea,Standard Double Mountain,10.0,Solo traveler,"[is, it, necessary, to, write, a, review, for,..."
1,"As expected, Shilla is Shilla!! Adding the eve...",Stayed 1 night in November 2023,Seo from South Korea,Deluxe Double Mountain,10.0,Couple,"[as, expected, shilla, is, shilla, adding, the..."
2,Exellent Location & Service”\nWe stayed at Shi...,Stayed 3 nights in October 2023,Aya from United States,Deluxe Double Mountain,10.0,Couple,"[exellent, location, service, we, stayed, at, ..."
3,"""I can understand why many people are looking ...",Stayed 2 nights in October 2023,sang from South Korea,Standard Twin Mountain,10.0,Couple,"[i, can, understand, why, many, people, are, l..."
4,완벽산 신라호텔 경험”\n가든뷰에 17개월 아기와 함께 묵었습니다. 비록 저녁에 윗...,Stayed 1 night in September 2023,SEUNGJIN from South Korea,Premier Terrace Double Garden,10.0,Family with young children,[17]
...,...,...,...,...,...,...,...
2595,A very nice resort”\nMake you feel great once ...,Stayed 2 nights in October 2016,"wai from Hong Kong SAR, China",NaN,9.2,Family with teens,NaN
2596,The hotel rooms update is way over due.”\nThe ...,Stayed 5 nights in October 2016,David from United States,NaN,6.4,Family with teens,NaN
2597,Hotel with great children facilities”\nThis ho...,Stayed 3 nights in June 2016,Chye from Singapore,NaN,9.2,Family with young children,NaN
2598,Perfect Hotel for Relaxing”\nThe hotel offers ...,Stayed 2 nights in September 2016,suryati from Indonesia,NaN,9.2,Group,NaN
